In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/glove-twitter/glove.twitter.27B.25d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.50d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.200d.txt
/kaggle/input/glove-twitter/glove.twitter.27B.100d.txt


In [2]:
# Load the competetion datasets

class KaggleReader(object):
    def __init__(self):
        self.df_objects = {}        
    def read_kaggle_df(self, df_name, df_path):
        self.df_objects[df_name] = pd.read_csv(df_path)
        print()
        print(df_name + ' loaded')
        print("Shape => " + str(self.df_objects[df_name].shape))
    
df_objs = KaggleReader()
df_objs.read_kaggle_df('train', "../input/tweet-sentiment-extraction/train.csv")
df_objs.read_kaggle_df('test', "../input/tweet-sentiment-extraction/test.csv")
df_objs.read_kaggle_df('submission', "../input/tweet-sentiment-extraction/sample_submission.csv")


train loaded
Shape => (27481, 4)

test loaded
Shape => (3534, 3)

submission loaded
Shape => (3534, 2)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from numpy import hstack, vstack
# Tokenize text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from nltk.tokenize import WordPunctTokenizer

# Model
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Embedding, Dropout, Input, Bidirectional, TimeDistributed
from keras.initializers import Constant
import keras

from sklearn.preprocessing import OneHotEncoder, StandardScaler

Using TensorFlow backend.


In [4]:
from keras.layers.merge import concatenate

In [169]:
THRESHOLD = 0.5

In [197]:
class Handlers(object):
    def __init__(self):
        self.run_tests()
        pass
    
    def draw_pr_curve_plt(self, Y_valid, y_pred, x_range=1.0):
    #     (precision, recall, x_range=1.0):

            precision, recall, thresholds_pr = precision_recall_curve(Y_valid, y_pred)

            # import dependencies
            import matplotlib.pyplot as plt

            plt.step(recall, precision, color='b', alpha=0.2,
                     where='post')
            plt.fill_between(recall, precision, alpha=0.2, color='b')
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, x_range])
            plt.show()
     
    def draw_roc_curve_plt(self, Y_valid, y_pred):
#     (fpr, tpr, auc):
        # import dependencies
        import matplotlib.pyplot as plt
        fpr, tpr, thresholds_roc = roc_curve(Y_valid, y_pred)
        auc_roc = auc(fpr, tpr)
        
        plt.figure(1)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(fpr, tpr, label='(area = {:.3f})'.format(auc_roc))
        # plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
        plt.xlabel('False positive rate')
        plt.ylabel('True positive rate')
        plt.title('ROC curve')
        plt.legend(loc='best')
        plt.show()

    # Optimize using KMP
    def return_start_end_indices(self, big, small):
        if len(small) <= 0 or len(big) <= 0:
            return (-1,-1)
        i = 0
        j = 0
        started = False
        while i < len(big):
            if big[i] == small[j]:
                if started == False:
                    started = True
                    i_start = i

                if j == len(small) - 1:
                    return (i-len(small)+1, i)
                j += 1

            else:
                if started == True:
                    started = False
                    i = i_start + 1
                j = 0
            i += 1
        return (-1,-1)
    
    def run_tests(self):
        
        assert self.return_start_end_indices(['i', '`', 'd', 'have', 'responded', ',', 'if', 'i', 'were', 'going'], ['have', 'responded', ',']) == (3,5)
        assert self.return_start_end_indices('abc', '') == (-1,-1)
        assert self.return_start_end_indices('abc', 'a') == (0,0)
        assert self.return_start_end_indices('abc', 'b') == (1,1)
        assert self.return_start_end_indices('abc', 'c') == (2,2)
        assert self.return_start_end_indices('abc', 'ab') == (0,1)
        assert self.return_start_end_indices('abc', 'bc') == (1,2)
        assert self.return_start_end_indices('abc', 'ac') == (-1,-1)
        assert self.return_start_end_indices('abc', 'abc') == (0,2)
        assert self.return_start_end_indices('abcabcabc', 'abc') == (0,2)
        assert self.return_start_end_indices('ababcc', 'abc') == (2,4)
    
    def jaccard(self, str1, str2): 
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    
    def load_gloVe_embeddings(self):
        # Boilerplate taken from here - https://www.kaggle.com/stacykurnikova/using-glove-embedding
        embeddings_index = {}
        f = open('/kaggle/input/glove-twitter/glove.twitter.27B.25d.txt')
        for line in f:
            values = line.split(' ')
            word = values[0] ## The first entry is the word
            coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
            embeddings_index[word] = coefs
        f.close()
        print('GloVe data loaded')
        return embeddings_index
    
    def load_embeddings_matrix(self, embeddings_index, index_tokenizer):
        # https://www.kaggle.com/stacykurnikova/using-glove-embedding
        # Create an embedding matrix with embedding vectors for the tokens recognized in the vocab of tweets
        
        EMBEDDING_DIM = embeddings_index.get('a').shape[0]
        # num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
        num_words = len(index_tokenizer.word_index) + 1

        # To Do: constrain the vocab size
        embedding_matrix = np.random.uniform(-1,+1,(num_words, EMBEDDING_DIM))
        count_in_embedding_vocab = 0
        for word, i in index_tokenizer.word_index.items():
            embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
                count_in_embedding_vocab += 1
#         TO DO:
#             else:
#                 PASS EMBEDDINGS OF SYNONYMS (!!! == !)
        return embedding_matrix
        # print(count_in_embedding_vocab)
        # print(len(embedding_matrix))
    
    def get_token_indices(self, x):
        span_generator = WordPunctTokenizer().span_tokenize(x)
        spans = [span for span in span_generator]
        return spans
    
    # TO DO: compensate for end and start on top of padding
    def get_pred_text_span(self, x):
        start_token_index = int(x['pred_start'])
        end_token_index = int(x['pred_end'])
        token_indices_list = x['tokens_indices']
        text = x['text']

        # start token ind > end token ind (could change logic to also have either 0:end)
        if start_token_index > end_token_index:
            return text

        # start token ind and end token ind within bounds
        elif start_token_index < len(token_indices_list) and end_token_index < len(token_indices_list):
            return text[token_indices_list[start_token_index][0]: token_indices_list[end_token_index][1]]


        # start token ind after bounds (could change logic to also have either 0:end or sth similar)
        elif start_token_index >= len(token_indices_list):
            return text

        # only end token ind out of bounds
        elif start_token_index < len(token_indices_list) and end_token_index >= len(token_indices_list):
            return text[token_indices_list[start_token_index][0]: len(text)-1]
        
    def get_preds_out(self, preds_indexes, test_df):
        temp_df = pd.concat([test_df, pd.DataFrame(preds_indexes, columns=['pred_start', 'pred_end'])], axis=1)
        temp_df['tokens_indices'] = temp_df['trans_text'].apply(handlers.get_token_indices)
        # print((temp_df['tokens_indices'].apply(len) == temp_df['tokens'].apply(len)).value_counts())

        # Get the predictions
        return temp_df.apply(handlers.get_pred_text_span, axis = 1)
    
    def get_indexes_from_argmax(self, preds, test_df):
        start_preds = preds[0]
        end_preds = preds[1]
        start_preds = start_preds.reshape((start_preds.shape[0], start_preds.shape[1]))
        end_preds = end_preds.reshape((end_preds.shape[0], end_preds.shape[1]))
        start_inds = start_preds.argmax(axis=1)
        end_inds = end_preds.argmax(axis=1)
        return vstack([start_inds, end_inds]).transpose()
        
        
    def batch_jaccard(self, preds, test_df):
        temp_df = pd.concat([test_df, pd.DataFrame(preds, columns=['pred_start', 'pred_end'])], axis=1)
        temp_df['tokens_indices'] = temp_df['trans_text'].apply(handlers.get_token_indices)
        # print((temp_df['tokens_indices'].apply(len) == temp_df['tokens'].apply(len)).value_counts())

        # Get the predictions
        temp_df['out_pred_span'] = temp_df.apply(handlers.get_pred_text_span, axis = 1)
        return temp_df.apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean()
    
    
    def batch_jaccard_from_argmax(self, preds, test_df):
        start_preds = preds[0]
        end_preds = preds[1]
        start_preds = start_preds.reshape((start_preds.shape[0], start_preds.shape[1]))
        end_preds = end_preds.reshape((end_preds.shape[0], end_preds.shape[1]))
        start_inds = start_preds.argmax(axis=1)
        end_inds = end_preds.argmax(axis=1)
        return self.batch_jaccard(vstack([start_inds, end_inds]).transpose(), test_df)    

    def batch_jaccard_from_continuous_span(self, preds, test_df, cont_span_method='get_min_max_span_indices'):
        return self.batch_jaccard(
            handlers.get_pos_indices_continuous_span(preds, cont_span_method), 
            test_df
        )        
    


    def get_pos_indices_multi_output(self, preds):
        return (preds[:,1]>THRESHOLD)

    def get_min_max_span_indices(self, preds):
        pos_preds_multi_output = self.get_pos_indices_multi_output(preds)
    #     print(pos_preds_multi_output)
        out_min, out_max = MAX_SEQUENCE_LENGTH+1, -1
        for i in range(len(pos_preds_multi_output)):
            if pos_preds_multi_output[i] == True:
                out_min = min(out_min, i)
                out_max = max(out_max, i)
        # check validity
        if out_min>out_max:return((-1,-1))
        else: return((out_min, out_max))

    def get_max_continuous_span_indices(self, preds):
        pos_preds_multi_output = self.get_pos_indices_multi_output(preds)
        left, right = -1, -2
        i = 0
        while i < len(pos_preds_multi_output):
            if pos_preds_multi_output[i]==True:
                temp_left = i
                while (i < len(pos_preds_multi_output)) and (pos_preds_multi_output[i] == True):
                    temp_right = i
                    i+=1    
                if temp_right-temp_left > right - left:
                    right = temp_right
                    left = temp_left
            i+=1
        if right-left < 0: return (-1,-1)
        else: return((left, right))

    def get_pos_indices_continuous_span(self, preds, method='get_min_max_span_indices'):
        arr = np.empty((0,2), int)
        if method == 'get_min_max_span_indices':
            func = self.get_min_max_span_indices
        elif method == "get_max_continuous_span_indices":
            func = self.get_max_continuous_span_indices
        for i in range(len(preds)):
            row = func(preds[i])
            arr = vstack([arr, np.array([row[0], row[1]])])
        return arr

        # get_pos_indices_continuous_span(preds, 'get_min_max_span_indices')

handlers = Handlers()


# To add Jaccard Similarity Value on Validation set after each epoch
class Metrics(keras.callbacks.Callback):
    # https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, X_sentiment, y_val = self.validation_data[0], self.validation_data[1], self.validation_data[2]
        y_predict = np.asarray(self.model.predict([X_val, X_sentiment]))
        print('Val Jaccard Similarity: {}'.format(handlers.batch_jaccard(y_predict, test_df))) 
        return

    def get_data(self):
        return self._data
    
    
class MetricsCategorical(keras.callbacks.Callback):
    # https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, X_numerics, y_val = self.validation_data[0], self.validation_data[1], self.validation_data[2]
        y_predict = np.asarray(self.model.predict([X_val, X_numerics]))
        print('Val Jaccard Similarity: {}'.format(handlers.batch_jaccard_from_argmax(y_predict, test_df))) 
        return

    def get_data(self):
        return self._data    

class MetricsContinuousSpan(keras.callbacks.Callback):
    # https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, X_numerics, y_val = self.validation_data[0], self.validation_data[1], self.validation_data[2]
        y_predict = np.asarray(self.model.predict([X_val, X_numerics]))
        print('Val Jaccard Similarity (min_max): {}'.format(handlers.batch_jaccard_from_continuous_span(y_predict, test_df, 'get_min_max_span_indices'))) 
        print('Val Jaccard Similarity (max_con): {}'.format(handlers.batch_jaccard_from_continuous_span(y_predict, test_df, 'get_max_continuous_span_indices'))) 
        return

    def get_data(self):
        return self._data
    
metrics = Metrics()
metrics_categorical = MetricsCategorical()
metrics_continuous_span = MetricsContinuousSpan()

In [6]:
MAX_SEQUENCE_LENGTH = 40

class DataManipulationPipeline(object):
    def __init__(self):
        # Load dependencies
        try:
            self.handlers = handlers
        except NameError:
            self.handlers = Handlers()
            
        # init vars
        self.vars = {
            'MAX_SEQUENCE_LENGTH' : MAX_SEQUENCE_LENGTH
        }

    def pre_fit(self, X):
        
        # Strip the text
        X['text'] = X['text'].str.strip()
        # Lower case the text
        X['trans_text'] = X['text'].apply(str.lower)
        return X

    def create_sequences(self, X):
        # Tokenize the word tokens to word_indexes
        sequences = self.vars['keras_index_tokenizer'].texts_to_sequences(X['tokens'])
        # Pad the sequences to be fed to NN [Note that this will effectively change the start, end index if padded on post]
        sequences_padded = pad_sequences(sequences, maxlen=self.vars['MAX_SEQUENCE_LENGTH'], padding='post', truncating='post')        

        return sequences_padded
    
    def create_Y(self, X):
        X['trans_selected_text'] = X['selected_text'].apply(str.lower)
        X = self.punct_tokenize(X, 'trans_selected_text', 'tokens_selected_text', self.vars['punct_tokenizer'])
        X['start_end_indices'] = X.apply(lambda x: handlers.return_start_end_indices(x['tokens'], x['tokens_selected_text']), axis=1)
        # truncate the start end indices to end of MAX LEN of sequence
        X['start_end_indices'] = X['start_end_indices'].apply(lambda x: (x[0], (MAX_SEQUENCE_LENGTH-1) if x[1]>= MAX_SEQUENCE_LENGTH else x[1]))
        
        X = X[(X['start_end_indices'] != (-1,-1))]
        X = X.reset_index(drop=True)
        X['start_ind'] = X['start_end_indices'].apply(lambda x: x[0])
        X['end_ind'] = X['start_end_indices'].apply(lambda x: x[1])
        Y = hstack(
            (
                X['start_ind'].values.reshape(X.shape[0],1),
                X['end_ind'].values.reshape(X.shape[0],1)
            )
        )
        return Y,X
    
    def create_Y_Categorical(self, Y):
        Y_categorical = np.array([[0]*MAX_SEQUENCE_LENGTH for j in range(Y.shape[0])])
        for i in range(Y.shape[0]):
            for j in range(Y[i][0], Y[i][1]+1):
                Y_categorical[i][j] = 1
        
        return keras.utils.to_categorical(Y_categorical, 2)
        
    
    def handle_sentiment_feature_eng(self, X):
        sentiment_transform = self.vars['sentiment_one_hot'].transform(X['sentiment'].values.reshape((X['sentiment'].shape[0],1)))
        # Copy over each data row for MAX_SEQUENCE_LENGTH times to send it inside each LSTM sequence
#         sentiment_transform_repeated = np.array([([sentiment_transform[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
        
        # To Do: Scale?
        return sentiment_transform
    
    def text_stats(self, X):
        X['char_len'] = X['trans_text'].apply(len)
        X['word_len'] = X['tokens'].apply(len)
        X['char_word_ratio'] = X['char_len']/X['word_len']
            
        text_stats_singular = X[['char_len', 'word_len', 'char_word_ratio']].values
        
        # Standardize
#         text_stats_singular        

#         text_stats_repeated = np.array([([text_stats_singular[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
        return text_stats_singular

    def repeater(self, X):
        return np.array([([X[i] for x in range(MAX_SEQUENCE_LENGTH)]) for i in range(X.shape[0])])
    
        
    
    # Fits the parameters on train
    def fit_transform(self, X):
        
        # {x}
        # Remove nulls from train
        X = X.dropna()
        
        X = self.pre_fit(X)
        
        # Create word tokens from sentences using NTLK
        self.vars['punct_tokenizer'] = WordPunctTokenizer()
        # put sth else inseatd of X foe expanded vocab?
        X = self.punct_tokenize(X, 'trans_text', 'tokens', self.vars['punct_tokenizer'])

        # Fit Index-Tokenizer the vocab using Keras
        self.vars['keras_index_tokenizer'] = Tokenizer()
        # Use all words for extended voacb
        self.vars['keras_index_tokenizer'].fit_on_texts(
            pd.concat(
            [
                df_objs.df_objects['train']['text'].dropna().str.strip().apply(str.lower),
                df_objs.df_objects['test']['text'].dropna().str.strip().apply(str.lower)
            ]
        ).reset_index(drop=True).apply(WordPunctTokenizer().tokenize)
        )
        # len(keras_tokenizer.word_index)
        
        # load glove embeddings
        self.vars['embeddings_index'] = self.handlers.load_gloVe_embeddings() 
        self.vars['embeddings_matrix'] = self.handlers.load_embeddings_matrix(self.vars['embeddings_index'], self.vars['keras_index_tokenizer'])

        # {y}
        # Create label column - Word Tokenize selected_text, and create label indices
        Y,X = self.create_Y(X)
        Y_categorical = self.create_Y_Categorical(Y)
        # Run final X transform after Y since Y transform filters out some X
        sequences_padded = self.create_sequences(X)
        
        # Fit one hot encoder for sentiment (Try standard scalar downstream as well?)
        self.vars['sentiment_one_hot'] = OneHotEncoder(handle_unknown='ignore', sparse=False)
        self.vars['sentiment_one_hot'].fit(X['sentiment'].values.reshape((X['sentiment'].shape[0],1)))
        # print(enc.get_feature_names())
        X_sentiment_one_hot = self.handle_sentiment_feature_eng(X)
        
        # Create text stats
        X_text_stats = self.text_stats(X)
        self.vars['X_text_stats_standard_scaler'] = StandardScaler()
        self.vars['X_text_stats_standard_scaler'].fit(X_text_stats)
        X_text_stats_scaled = self.vars['X_text_stats_standard_scaler'].transform(X_text_stats)
        X_agg_numerics_all = hstack([X_sentiment_one_hot, X_text_stats_scaled])
        
        X_numerics_repeated = self.repeater(X_agg_numerics_all)
        
        return (sequences_padded, X_numerics_repeated, Y_categorical, X)


    # Transforms using the parameters on train
    def transform(self, X):
        
        X = self.pre_fit(X)
        
        # Create word tokens
        X = self.punct_tokenize(X, 'trans_text', 'tokens', self.vars['punct_tokenizer'])
        
        Y = None
        Y_categorical = None
        if 'selected_text' in X.columns:
            Y,X = self.create_Y(X)
            Y_categorical = self.create_Y_Categorical(Y)
        sequences_padded = self.create_sequences(X)
        X_sentiment_one_hot = self.handle_sentiment_feature_eng(X)        
        # Create text stats
        X_text_stats = self.text_stats(X)
        X_text_stats_scaled = self.vars['X_text_stats_standard_scaler'].transform(X_text_stats)
        
        X_agg_numerics_all = hstack([X_sentiment_one_hot, X_text_stats_scaled])
        X_numerics_repeated = self.repeater(X_agg_numerics_all)
    
        
        return (sequences_padded, X_numerics_repeated, Y_categorical, X)
        
        
    def punct_tokenize(self, X,old_col, new_col, punct_tokenizer):
        X[new_col] = X[old_col].apply(punct_tokenizer.tokenize)
        return X
        
        



# Get the optimum number of word length for sequence model - 40
# plt.hist(df_objs.df_objects['train']['tokens'].apply(len), bins=100)
# plt.show()
    
        
        

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_objs.df_objects['train'], test_size=0.15, random_state=42)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
# X_train, X_test, Y_train, Y_test, idx1, idx2 = train_test_split(df_objs.df_objects['train'], Y, np.arange(Y.shape[0]), test_size=0.15, random_state=42)


In [8]:
data_mainpulation_pipeline = DataManipulationPipeline()
(train_manip_X, train_numerics_repeated, train_manip_Y, train_df) = data_mainpulation_pipeline.fit_transform(train)
(test_manip_X, test_numerics_repeated, test_manip_Y, test_df) = data_mainpulation_pipeline.transform(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

GloVe data loaded


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [9]:
# train_manip_Y.shape
# train_df


    
# [0]*MAX_SEQUENCE_LENGTH


# [j for j in range(11,22)]
# train_manip_Y
# import matplotlib.pyplot as plt
# plt.hist(train_df.tokens.apply(len), bins=100)
# plt.show()

In [10]:
# (train_manip_X, train_numerics_repeated, train_manip_Y, train_df)
# test_df
# test_manip_Y[1]

# print(test_manip_Y[2])

test_manip_Y.shape
# print([2])

(3804, 40, 2)

In [11]:
VOCAB_SIZE = len(data_mainpulation_pipeline.vars['keras_index_tokenizer'].word_index)+1
EMBEDDING_DIM = len(data_mainpulation_pipeline.vars['embeddings_index']['a'])

In [198]:
# Model
# ----------------------------------------------------
inputLayer_words = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputLayer_agg_numerics = Input(shape=(
    MAX_SEQUENCE_LENGTH,
    train_numerics_repeated.shape[2]
))

# Embedding layer for the tap names
wordEmbeddings = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputLayer_words)
wordEmbeddings = Embedding(
    input_dim=VOCAB_SIZE, 
    output_dim=EMBEDDING_DIM, 
    input_length=MAX_SEQUENCE_LENGTH, 
    weights=[data_mainpulation_pipeline.vars['embeddings_matrix']],
    trainable=False)(inputLayer_words)
# 
merged_Input = concatenate([wordEmbeddings, inputLayer_agg_numerics])
# inputLayer_agg_indices
# # LSTM
lstm_1 = Bidirectional(LSTM(50, 
                            return_sequences = True,
                            dropout=0.3,
                            recurrent_dropout=0.3,
                           ))(merged_Input)
# input_shape=(25,(EMBEDDING_DIM))
td = TimeDistributed(Dense(25))(lstm_1)


# drp1 = Dropout(0.2)(lstm_1)

# # Dense
# dense_0 = Dense(75, activation='relu')(drp1)
# drp2 = Dropout(0.2)(dense_0)
# dense_1 = Dense(30, activation='relu')(drp2)
outputLayer = Dense(2, activation='softmax')(td)

model_2 = Model(inputs=[inputLayer_words, inputLayer_agg_numerics], outputs=outputLayer)

model_2.compile(loss='categorical_crossentropy', optimizer='adam')
# model.compile(loss='mse', optimizer='adam', metrics=[jaccard])
model_2.summary()
# ----------------------------------------------------

model_2.fit([train_manip_X, train_numerics_repeated], train_manip_Y, validation_data=([test_manip_X, test_numerics_repeated], test_manip_Y), batch_size=100, epochs=200, callbacks=[metrics_continuous_span])

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 40, 25)       728175      input_11[0][0]                   
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 40, 6)        0                                            
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 40, 31)       0           embedding_14[0][0]               
                                                                 input_12[0][0]             

21561/21561 [==============================] - 11s 523us/step - loss: 0.1533 - val_loss: 0.1315
Val Jaccard Similarity (min_max): 0.646511873058198
Val Jaccard Similarity (max_con): 0.6338391444929203
Epoch 31/200
21561/21561 [==============================] - 11s 512us/step - loss: 0.1525 - val_loss: 0.1321
Val Jaccard Similarity (min_max): 0.6564103254132793
Val Jaccard Similarity (max_con): 0.6431578457879716
Epoch 32/200
21561/21561 [==============================] - 12s 536us/step - loss: 0.1537 - val_loss: 0.1315
Val Jaccard Similarity (min_max): 0.636838118451814
Val Jaccard Similarity (max_con): 0.6266106977923461
Epoch 33/200
21561/21561 [==============================] - 11s 509us/step - loss: 0.1525 - val_loss: 0.1319
Val Jaccard Similarity (min_max): 0.6363187326706004
Val Jaccard Similarity (max_con): 0.626037153433276
Epoch 34/200
21561/21561 [==============================] - 11s 512us/step - loss: 0.1515 - val_loss: 0.1306
Val Jaccard Similarity (min_max): 0.64436760697

21561/21561 [==============================] - 11s 529us/step - loss: 0.1472 - val_loss: 0.1291
Val Jaccard Similarity (min_max): 0.6720212635965314
Val Jaccard Similarity (max_con): 0.6598558480700073
Epoch 69/200
21561/21561 [==============================] - 11s 511us/step - loss: 0.1493 - val_loss: 0.1293
Val Jaccard Similarity (min_max): 0.6718129504004594
Val Jaccard Similarity (max_con): 0.6596569935932761
Epoch 70/200
21561/21561 [==============================] - 11s 508us/step - loss: 0.1479 - val_loss: 0.1294
Val Jaccard Similarity (min_max): 0.6711264661101918
Val Jaccard Similarity (max_con): 0.6601568739057176
Epoch 71/200
21561/21561 [==============================] - 11s 508us/step - loss: 0.1471 - val_loss: 0.1291
Val Jaccard Similarity (min_max): 0.6672147381749353
Val Jaccard Similarity (max_con): 0.6563333804867292
Epoch 72/200
21561/21561 [==============================] - 12s 547us/step - loss: 0.1476 - val_loss: 0.1290
Val Jaccard Similarity (min_max): 0.67123775

21561/21561 [==============================] - 11s 511us/step - loss: 0.1458 - val_loss: 0.1292
Val Jaccard Similarity (min_max): 0.6709395701752333
Val Jaccard Similarity (max_con): 0.6603161697542057
Epoch 107/200
21561/21561 [==============================] - 11s 517us/step - loss: 0.1471 - val_loss: 0.1299
Val Jaccard Similarity (min_max): 0.6760311432387912
Val Jaccard Similarity (max_con): 0.6676341516364879
Epoch 108/200
21561/21561 [==============================] - 11s 527us/step - loss: 0.1459 - val_loss: 0.1290
Val Jaccard Similarity (min_max): 0.6762085128727864
Val Jaccard Similarity (max_con): 0.6657549263809303
Epoch 109/200
21561/21561 [==============================] - 11s 514us/step - loss: 0.1454 - val_loss: 0.1295
Val Jaccard Similarity (min_max): 0.6743331428475724
Val Jaccard Similarity (max_con): 0.6650239791939716
Epoch 110/200
21561/21561 [==============================] - 11s 506us/step - loss: 0.1453 - val_loss: 0.1336
Val Jaccard Similarity (min_max): 0.6805

21561/21561 [==============================] - 13s 597us/step - loss: 0.1423 - val_loss: 0.1301
Val Jaccard Similarity (min_max): 0.6684419050483369
Val Jaccard Similarity (max_con): 0.6592006167581083
Epoch 145/200
21561/21561 [==============================] - 14s 634us/step - loss: 0.1435 - val_loss: 0.1301
Val Jaccard Similarity (min_max): 0.6739790359572941
Val Jaccard Similarity (max_con): 0.6645658649064417
Epoch 146/200
21561/21561 [==============================] - 11s 518us/step - loss: 0.1424 - val_loss: 0.1300
Val Jaccard Similarity (min_max): 0.6736616703247419
Val Jaccard Similarity (max_con): 0.6627213939301647
Epoch 147/200
21561/21561 [==============================] - 11s 504us/step - loss: 0.1430 - val_loss: 0.1307
Val Jaccard Similarity (min_max): 0.6797850052414686
Val Jaccard Similarity (max_con): 0.6670833732015576
Epoch 148/200
21561/21561 [==============================] - 12s 545us/step - loss: 0.1440 - val_loss: 0.1304
Val Jaccard Similarity (min_max): 0.6784

21561/21561 [==============================] - 11s 508us/step - loss: 0.1423 - val_loss: 0.1312
Val Jaccard Similarity (min_max): 0.6797915670142605
Val Jaccard Similarity (max_con): 0.6699707014969868
Epoch 183/200
21561/21561 [==============================] - 11s 512us/step - loss: 0.1430 - val_loss: 0.1358
Val Jaccard Similarity (min_max): 0.6805810031533864
Val Jaccard Similarity (max_con): 0.6741390931985577
Epoch 184/200
21561/21561 [==============================] - 12s 572us/step - loss: 0.1421 - val_loss: 0.1303
Val Jaccard Similarity (min_max): 0.6787083334444827
Val Jaccard Similarity (max_con): 0.6677811349021833
Epoch 185/200
21561/21561 [==============================] - 11s 515us/step - loss: 0.1407 - val_loss: 0.1302
Val Jaccard Similarity (min_max): 0.6766151623173279
Val Jaccard Similarity (max_con): 0.664034095915537
Epoch 186/200
21561/21561 [==============================] - 11s 508us/step - loss: 0.1431 - val_loss: 0.1293
Val Jaccard Similarity (min_max): 0.67290

Epoch 98/100
21561/21561 [==============================] - 8s 357us/step - loss: 26.4123 - val_loss: 32.8301
Val Jaccard Similarity: 0.5266293328859964

Epoch 99/100
21561/21561 [==============================] - 8s 356us/step - loss: 26.0379 - val_loss: 32.5830
Val Jaccard Similarity: 0.5592650588400309

Epoch 100/100
21561/21561 [==============================] - 8s 352us/step - loss: 25.8462 - val_loss: 32.7917
Val Jaccard Similarity: 0.5504962474510444

In [ ]:
model_2.fit([train_manip_X, train_numerics_repeated], train_manip_Y, validation_data=([test_manip_X, test_numerics_repeated], test_manip_Y), batch_size=1000, epochs=200, callbacks=[metrics_continuous_span])

Train on 21561 samples, validate on 3804 samples
Epoch 1/200
21561/21561 [==============================] - 9s 396us/step - loss: 0.1415 - val_loss: 0.1301
Val Jaccard Similarity (min_max): 0.6743554051905798
Val Jaccard Similarity (max_con): 0.6643664700180647
Epoch 2/200
21561/21561 [==============================] - 8s 394us/step - loss: 0.1402 - val_loss: 0.1305
Val Jaccard Similarity (min_max): 0.67802883279624
Val Jaccard Similarity (max_con): 0.6684265132922482
Epoch 3/200
21561/21561 [==============================] - 8s 373us/step - loss: 0.1409 - val_loss: 0.1308
Val Jaccard Similarity (min_max): 0.6783518547600192
Val Jaccard Similarity (max_con): 0.6689032508290057
Epoch 4/200
21561/21561 [==============================] - 8s 355us/step - loss: 0.1406 - val_loss: 0.1308
Val Jaccard Similarity (min_max): 0.6773932001678056
Val Jaccard Similarity (max_con): 0.6685068957267316
Epoch 5/200
21561/21561 [==============================] - 8s 354us/step - loss: 0.1402 - val_loss: 0

21561/21561 [==============================] - 8s 374us/step - loss: 0.1395 - val_loss: 0.1309
Val Jaccard Similarity (min_max): 0.6805614149705306
Val Jaccard Similarity (max_con): 0.6721356002920762
Epoch 40/200
21561/21561 [==============================] - 8s 351us/step - loss: 0.1406 - val_loss: 0.1312
Val Jaccard Similarity (min_max): 0.680963378066963
Val Jaccard Similarity (max_con): 0.6723138288575291
Epoch 41/200
21561/21561 [==============================] - 8s 350us/step - loss: 0.1392 - val_loss: 0.1313
Val Jaccard Similarity (min_max): 0.6797231404075753
Val Jaccard Similarity (max_con): 0.6714707208614238
Epoch 42/200
21561/21561 [==============================] - 7s 344us/step - loss: 0.1387 - val_loss: 0.1313
Val Jaccard Similarity (min_max): 0.6796524548810761
Val Jaccard Similarity (max_con): 0.6710513459370263
Epoch 43/200
21561/21561 [==============================] - 7s 345us/step - loss: 0.1392 - val_loss: 0.1312
Val Jaccard Similarity (min_max): 0.68020914644805

21561/21561 [==============================] - 8s 352us/step - loss: 0.1396 - val_loss: 0.1310
Val Jaccard Similarity (min_max): 0.6800638644691653
Val Jaccard Similarity (max_con): 0.6710767765288648
Epoch 78/200
21561/21561 [==============================] - 7s 344us/step - loss: 0.1394 - val_loss: 0.1310
Val Jaccard Similarity (min_max): 0.6816998815101002
Val Jaccard Similarity (max_con): 0.671495506529113
Epoch 79/200
21561/21561 [==============================] - 7s 343us/step - loss: 0.1399 - val_loss: 0.1311
Val Jaccard Similarity (min_max): 0.6813746594439845
Val Jaccard Similarity (max_con): 0.6710469577848462
Epoch 80/200
21561/21561 [==============================] - 7s 348us/step - loss: 0.1403 - val_loss: 0.1310
Val Jaccard Similarity (min_max): 0.67737544241109
Val Jaccard Similarity (max_con): 0.6686268503013361
Epoch 81/200
21561/21561 [==============================] - 8s 348us/step - loss: 0.1402 - val_loss: 0.1310
Val Jaccard Similarity (min_max): 0.6799629323334349

21561/21561 [==============================] - 7s 348us/step - loss: 0.1393 - val_loss: 0.1315
Val Jaccard Similarity (min_max): 0.6806186487749013
Val Jaccard Similarity (max_con): 0.6695265927266697
Epoch 116/200
21561/21561 [==============================] - 8s 358us/step - loss: 0.1391 - val_loss: 0.1309
Val Jaccard Similarity (min_max): 0.675686189500844
Val Jaccard Similarity (max_con): 0.6658109757006183
Epoch 117/200
21561/21561 [==============================] - 7s 346us/step - loss: 0.1387 - val_loss: 0.1320
Val Jaccard Similarity (min_max): 0.6826571535373332
Val Jaccard Similarity (max_con): 0.6743402804552623
Epoch 118/200
21561/21561 [==============================] - 8s 352us/step - loss: 0.1402 - val_loss: 0.1315
Val Jaccard Similarity (min_max): 0.6820062261361359
Val Jaccard Similarity (max_con): 0.6729870734420484
Epoch 119/200
21561/21561 [==============================] - 8s 361us/step - loss: 0.1400 - val_loss: 0.1316
Val Jaccard Similarity (min_max): 0.6803651283

In [ ]:
# model_2.fit([train_manip_X, train_manip_X_sentiment_one_hot_repeated], train_manip_Y, validation_data=([test_manip_X, test_manip_X_sentiment_one_hot_repeated], test_manip_Y), batch_size=5000, epochs=200, callbacks=[metrics])

In [ ]:
# model_2.fit([train_manip_X, train_manip_X_sentiment_one_hot_repeated], train_manip_Y, validation_data=([test_manip_X, test_manip_X_sentiment_one_hot_repeated], test_manip_Y), batch_size=2000, epochs=200, callbacks=[metrics])

In [13]:
preds = model_2.predict([test_manip_X])

# # test_X_df = df_objs.df_objects['train'].iloc[idx2].reset_index(drop=True)
# test_df_1 = pd.concat([test_df, pd.DataFrame(preds, columns=['pred_start', 'pred_end'])], axis=1)
# test_df_1['tokens_indices'] = test_df_1['trans_text'].apply(handlers.get_token_indices)
# print((test_df_1['tokens_indices'].apply(len) == test_df_1['tokens'].apply(len)).value_counts())

# # Get the predictions
# test_df_1['out_pred_span'] = test_df_1.apply(handlers.get_pred_text_span, axis = 1)

In [55]:
# test_manip_Y[0]
# test_manip_Y[1].shape
# preds.reshape((preds.shape[0], preds.shape[1]))
# [:,1].shape

# preds

array([ True, False,  True,  True,  True,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [65]:
i = 100
print(preds.shape)
print(test_df.iloc[i]['start_ind'])
print(test_df.iloc[i]['end_ind'])
print(preds[:,:,1][i]>0.5)
print(len(test_df.iloc[i]['tokens']))
# print(np.around(preds[i], decimals=2))

# Two Ways:
# 1) Min to Max
# 2) Max Continuous span

(3804, 40, 2)
0
9
[False  True False  True  True  True  True  True  True  True False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
10


In [164]:

# get_pos_indices_continuous_span(preds, 'get_min_max_span_indices')).sum()
    
        
# #         i+=1
      
    
# # get_min_max_span_indices(preds[0])    
# # print(test_df.iloc[22])
# # print(test_manip_Y[22])
# # print(get_max_continuous_span_indices(preds[22]))
# for i in range(30, 50):
#     print(get_max_continuous_span_indices(preds[i]))
#     print()
# vstack([np.array([1,2]), None])

array([[ 0,  6],
       [ 0, 17],
       [ 7, 10],
       ...,
       [-1, -1],
       [ 1,  7],
       [ 3,  3]])

In [189]:
THRESHOLD = 0.5
temp_out = pd.concat([
    handlers.get_preds_out(
        handlers.get_pos_indices_continuous_span(preds, 'get_min_max_span_indices'), 
#         handlers.get_pos_indices_continuous_span(preds, 'get_max_continuous_span_indices'), 
        test_df), 
    test_df['selected_text']],
    axis=1
)
temp_out.columns = ['predicted_text', 'selected_text']
# temp_out['predicted_text']
print(temp_out.apply(lambda x: handlers.jaccard(x['predicted_text'], x['selected_text']), axis=1).mean())

temp_out = pd.concat([
    handlers.get_preds_out(
#         handlers.get_pos_indices_continuous_span(preds, 'get_min_max_span_indices'), 
        handlers.get_pos_indices_continuous_span(preds, 'get_max_continuous_span_indices'), 
        test_df), 
    test_df['selected_text']],
    axis=1
)
temp_out.columns = ['predicted_text', 'selected_text']
# temp_out['predicted_text']
print(temp_out.apply(lambda x: handlers.jaccard(x['predicted_text'], x['selected_text']), axis=1).mean())


0.568592198881803
0.5277566266345529


- Check if embedding matrix is correctly created

In [ ]:
# print('Overall Jaccard from Model:')
# print(test_df_1[['text', 'selected_text', 'out_pred_span']].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Overall Jaccard baseline from predicting the complete text:')
# print(test_df_1[['text', 'selected_text', 'out_pred_span']].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())

# print('Overall Jaccard using model for NOT neutral and baseline for neutral:')
# print(pd.concat([
#     test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1),
#     test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1)
# ]).mean())


# print('Model Jaccard for neutral')
# print(test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Baseline Jaccard for neutral from predicting complete text')
# print(test_df_1[test_df_1['sentiment'] == 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())

# print('Model Jaccard for NOT neutral')
# print(test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean())

# print('Baseline Jaccard for NOT neutral from predicting complete text')
# print(test_df_1[test_df_1['sentiment'] != 'neutral'].apply(lambda x: handlers.jaccard(x['selected_text'], x['text']), axis=1).mean())



In [ ]:
# test_df_1.apply(lambda x: handlers.jaccard(x['selected_text'], x['out_pred_span']), axis=1).mean()

In [ ]:
# test_df_1[['text', 'selected_text', 'out_pred_span']]

In [ ]:
# test_df_1['selected_text'].values